This notebook contains some tests and comparisons to ensure that I have implemented the NuGrid yields correctly.

In [ ]:
import sys
sys.path.append("/astro/dboyea/source/")

In [ ]:
from NuPyCEE import omega
from NuPyCEE import sygma
import NuPyCEE
sygma = NuPyCEE.sygma

In [ ]:
import numpy as np
import pandas as pd
import arya
import matplotlib.pyplot as plt

import surp
import vice

In [ ]:
from VICE.vice.yields.agb.pignatari16.raw import process_safe

In [ ]:
s = sygma.sygma(iniZ=0.01)

In [ ]:
fryer = process_safe.read_element_yields("/astro/dboyea/source/VICE/vice/yields/agb/pignatari16/raw/element_yield_table_MESAonly_fryer12_delay_total.txt", process_safe.elements)

fryer = {
    ele: pd.DataFrame(fryer[ele], columns=["M", "Mrem", "Z", "y", "Z0"])
    for ele in fryer.keys()
}

In [ ]:
def plot_vice_table(ele="C", Z=0.02, color="none"):
    y, m, z = vice.yields.agb.grid(ele, study="pignatari16")

    i = next((i for i, zz in enumerate(z) if zz == Z), None)

    m = np.array(m)
    y = np.array([yy[i] for yy in y])
    Zini = vice.solar_z(ele) * Z / 0.016 
    m_ret = z_ini(ele, Z) * (m - m_rem(m))
    m_ej = y * m
    y_tot = m_ej + m_ret 

    plt.scatter(m, y_tot, s=15, facecolors='none', ec=color, marker="s")

    plt.yscale("log")


In [ ]:
y, m, z = vice.yields.agb.grid("c", study="pignatari16")

In [ ]:
def plot_fryer_table(ele, Z, **kwargs):
    df = fryer[ele.title()]
    filt = np.isclose(df.Z, Z)
    if sum(filt) == 0:
        print("filt failed for",ele, Z)
        return

    plt.scatter(df[filt].M, df[filt].y, **kwargs)

In [ ]:
def plot_all_yields(ele):
    for i, Z in enumerate([0.0001, 0.001, 0.006, 0.01, 0.02]):
        s.plot_table_yield([ele.title()], "mini", Z=Z)
        plt.xlim(0.5, 7.5)
        plot_vice_table(Z=Z, color=arya.COLORS[i], ele=ele)
        plot_fryer_table(ele, Z, color="red", zorder=100, s=1)
        
    arya.Legend(-1)

In [ ]:
def m_rem(m):
    return np.where(m >= 8, 1.44, 0.394 + 0.109 * m)

In [ ]:
def z_ini(ele, Z):
    filt = np.isclose(df.Z, Z)

    Z0 = fryer[ele.title()].Z0[filt].iloc[1]
    return  Z0

In [ ]:
def A(ele):
    return surp.gce_math.molmass.ELEMENTS[ele.title()].number

In [ ]:


for (i, Z) in enumerate([0.0001, 0.001,0.006, 0.01, 0.02]):
    xs = []
    ys = []

    for ele in process_safe.elements[4:]:
        x = A(ele)
        y = z_ini(ele,Z) * 0.016/Z / vice.solar_z(ele)
    
        xs.append(x)
        ys.append(y)
        
    plt.scatter(xs, ys, label=f"Z={Z}", marker=["o", "s", "*", "d", "+"][i], s=2*[5, 3, 1, 5, 3][i])



plt.xlabel("atomic number")
plt.ylabel(r" $Z_{\rm 0, NuGrid} (Z_\odot/Z)$ / $Z_{\odot, \rm VICE}$ ")
plt.title("ratio of solar scaled initial abundances")
arya.Legend(-1)


In [ ]:
plot_all_yields("c")
plt.ylim(1e-4, 0.1)


In [ ]:
plot_all_yields("n")
plt.ylim(1e-5, 0.1)


In [ ]:
plot_all_yields("o")
plt.ylim(1e-5, 0.1)


In [ ]:
plot_all_yields("F")
plt.ylim(1e-10, 1e-4)

In [ ]:
plot_all_yields("Ne")
plt.ylim(1e-5, 3e-2)

In [ ]:
plot_all_yields("Mg")

In [ ]:
plot_all_yields("Pb")

# unecessary

In [ ]:
def sygma_to_pd(s):
    dt = s.history.timesteps
    times = np.cumsum(dt)
    times = np.insert(times, 0, 0.)
    df = pd.DataFrame(s.history.ism_elem_yield, columns=s.history.elements)
    df["time"] = times

    return df

In [ ]:
def run_sygma(ele="C", Z=0.01, table="pignatari16"):
    if table == "pignatari16":
        table = "yield_tables/agb_and_massive_stars_nugrid_MESAonly_fryer12rapid.txt"
    elif table == "pignatari16b":
        table = "yield_tables/agb_and_massive_stars_nugrid_N13.txt"
    elif table == "karakas10":
        table = "yield_tables/agb_and_massive_stars_K10_LC18_R000.txt"
    elif table == "cristallo11":
        table = "yield_tables/agb_and_massive_stars_C15_LC18_R_mix.txt"
    elif table == "karakas10b":
        table = "yield_tables/agb_and_massive_stars_K10_LC18_R300.txt"
    else:
        raise Exception("unknown table", table)
        
    s = sygma.sygma(iniZ=Z,
                table=table,
                    
                tend=10e9, mgal=1, ns_merger_on=False,
                sn1a_on=False,
                scale_cor = [[1.0, 8], [0., 100]]
               )

    return s

In [ ]:
from vice.yields.agb import pignatari16

In [ ]:
vice.yields.ccsne.settings["C"] = 0

In [ ]:
def plot_vice(ele="C", Z=0.014, **kwargs):
    yields, times = vice.single_stellar_population(ele, Z=Z, time=10, mstar=1)

    times = [t*1e9 for t in times]
    plt.plot(times, yields, **kwargs)

In [ ]:
def run_sygma(ele="C", Z=0.014, table="pignatari16"):
    if table == "pignatari16":
        table = "yield_tables/agb_and_massive_stars_nugrid_MESAonly_fryer12rapid.txt"
    elif table == "pignatari16b":
        table = "yield_tables/agb_and_massive_stars_nugrid_N13.txt"
    elif table == "karakas10":
        table = "yield_tables/agb_and_massive_stars_K10_LC18_R000.txt"
    elif table == "cristallo11":
        table = "yield_tables/agb_and_massive_stars_C15_LC18_R_mix.txt"
    elif table == "karakas10b":
        table = "yield_tables/agb_and_massive_stars_K10_LC18_R300.txt"
    else:
        raise Exception("unknown table", table)
        
    s = sygma.sygma(iniZ=Z,
                table=table,
                    
                tend=10e9, mgal=1, ns_merger_on=False,
                sn1a_on=False,
                scale_cor = [[1.0, 8], [0., 100]]
               )

    return s

In [ ]:
def plot_sygma(ele="C", Z=0.014, table="pignatari16", **kwargs):

    s = run_sygma(ele, Z=Z, table=table)
    
    df = sygma_to_pd(s)

    t0 = 1e8
    idx0 = np.argmin(np.abs(df.time - t0))
    y = df[ele.title()]
    y -= y[idx0]
    plt.plot(df.time, df[ele.title()], **kwargs)


In [ ]:
vice.yields.agb.grid("c", "pignatari16")[2]

In [ ]:
vice.yields.ccsne.settings["C"] = 0

In [ ]:
kwargs_interp = dict(
    interp_kind="linear",
    min_mass=1,
    no_negative=False, 
)

In [ ]:
for Z in (0.0001, 0.001, 0.006, 0.01, 0.02):
    plot_sygma(Z=Z, label="SYGMA")
    plot_vice(Z=Z, label="VICE")
    plt.xscale("log")
    plt.yscale("log")
    plt.xlabel("time")
    plt.ylabel("M(C)(t) / Mstar")
    plt.xlim(1e6, 1e10)
    plt.legend()
    plt.show()

In [ ]:
from vice.yields.agb import karakas10

for Z in (0.0001, 0.004, 0.008, 0.02):
    plot_sygma(Z=Z, table="karakas10")
    plot_sygma(Z=Z, table="karakas10b")

    plot_vice(Z=Z)
    plt.xscale("log")
    plt.yscale("log")
    plt.xlabel("time")
    plt.ylabel("M(C)(t) / Mstar")
    plt.xlim(1e6, 1e10)
    plt.show()

In [ ]:
from vice.yields.agb import cristallo11

vice.yields.agb.settings["c"] = surp.agb_interpolator.interpolator("c", study = "cristallo11", **kwargs_interp
        )

for Z in (0.0001, 0.001, 0.006, 0.01, 0.02):
    plot_sygma(Z=Z, table="cristallo11")
    plot_vice(Z=Z)
    plt.xscale("log")
    plt.yscale("log")
    plt.xlabel("time")
    plt.ylabel("M(C)(t) / Mstar")
    plt.xlim(1e6, 1e10)
    plt.show()

In [ ]:
s = run_sygma(Z=0.01)

In [ ]:
s.plot_table_param(quantity="Ekin_wind")

In [ ]:
from NuPyCEE import read_yields

In [ ]:
s.plot_table_remnant(iniZ=0.02)

In [ ]:
s.plot_table_yield("mini", "C-12", 0.001)

In [ ]:
s.plot_mass_range_contributions("C")